In [8]:
import os
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [9]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 300)

In [10]:
## Connection
connection = presto.connect(
        host='presto-gateway.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='manoj.ravirajan@rapido.bike'
)## Parameter 

## Requirement 

### Userbase CSV for this Campaign


		a. Cities -> Hyderabad/Kolkata/Bangalore
        b. Condition → Users who do not have age present in our system and are 90->21 days active in above cities. 

In [13]:
## Userbase_Age_Campaign


Userbase_Age_Campaign = f"""

        WITH order_logs_snapshot AS (

            SELECT
                city_name,
                customer_id,
                customer_obj_mobile customer_mobile
            FROM 
                orders.order_logs_snapshot
            WHERE 
                yyyymmdd >= DATE_FORMAT(CURRENT_DATE - INTERVAL '21' DAY, '%Y%m%d') 
                AND city_name IN ('Hyderabad', 'Kolkata', 'Bangalore')
                AND service_obj_service_name IN ('Link', 'Auto') --, 'Auto Pool', 'CabEconomy')
                AND order_id IS NOT NULL
                AND order_status = 'dropped'
                AND (spd_fraud_flag is null or spd_fraud_flag=false)
            GROUP BY 1,2,3
            ),

            iallocator_customer_metrics AS (

            SELECT 
                run_date,
                customer_id,
                CASE 
                WHEN (cast(year_of_birth as int) BETWEEN 1940 AND 2015) THEN (2023 - cast(year_of_birth as int))
                END age

            FROM 
                datasets.iallocator_customer_metrics 
            WHERE 
                run_date = DATE_FORMAT( CURRENT_DATE - INTERVAL '2' day, '%Y-%m-%d')
            ),

            customer_metrics AS (

            select 
                CASE 
                WHEN age < 18 THEN 'Below 18'
                WHEN age >= 18 AND age <= 25 THEN '18 to 25'
                WHEN age >= 26 AND age <= 40 THEN '26 to 40'
                WHEN age >= 41 AND age <= 55 THEN '41 to 55'
                WHEN age > 55 THEN '55 +'
                WHEN age IS NULL THEN 'Unknown'
                END age,
                customer_id
            from 
                iallocator_customer_metrics
            ),

            customer_single_view AS (

            select 
                customerId 
            from 
                datasets.customer_single_view 
            where 
                dateOfBirth is null 
                and registrationCity IN ('hyderabad', 'kolkata', 'bangalore')
            )

            SELECT
                -- city_name,
                COALESCE(ols.customer_id, cm.customer_id, cs.customerId) customer_id,
                customer_mobile
            FROM 
                order_logs_snapshot ols 
            JOIN 
                customer_metrics cm 
                ON ols.customer_id = cm.customer_id
                AND age = 'Unknown' 

            JOIN 
                customer_single_view cs
                ON ols.customer_id = cs.customerId
            GROUP BY 1,2
"""

df_Userbase_Age_Campaign = pd.read_sql(Userbase_Age_Campaign, connection)
df_Userbase_Age_Campaign

,customer_id,customer_mobile
0,64fb030c588b8835b433ca3f,7675024468
1,6411414e3fdc1cb30a2b2231,8123563210
2,64f2f29234850c0e9a4610da,9844190433
3,61fa87606916b8b146b18e53,9573750907
4,61a60d220d37fd2d374b1c10,9676030266
...,...,...
1673000,6336aa48e90ec6448d7ad4e3,9538698894
1673001,615973e577c7e65990901667,7032089979
1673002,636c8f996a828365bde43e8c,9494229468
1673003,63287744747b4180e1b369ee,8919551890


In [14]:
df_Userbase_Age_Campaign.shape

(1673005, 2)

In [15]:
df_Userbase_Age_Campaign[['customer_id','customer_mobile']].to_csv('Userbase_Age_Campaign_Active_21days_Ban_Hyd_Kol.csv', index=False)

90 days gross customer - 8272625 <br>
90 days dropped customer - 6872783 <br>
21 days dropped customer - 1692599 <br>
14 days dropped customer - 1294295 <br>

In [ ]:
1687235-1294295